## 让训练更加稳当

- ### 目标：让梯度值在合理的范围内
    - 例如 [le-6, le3]
- ### 将乘法变加法
    - RestNet, LSTM
- ### 归一化
    - 梯度归一化，梯度剪裁
- ### 合理的权重初始化和激活函数

## 让每层的方差都是一个常数
- ### 建哥每层的输出和梯度都看成随机变量
- ### 让他们的均值和方差都保持一致

- 正向：
$$
\mathbb E[h_i^t] = 0 \\
\text{Var}[h_i^t] = a \quad \text{(方差a是一个常数)}
$$

- 反向：
$$
\mathbb E \left[ \frac{\partial \ell}{\partial h_i^t} \right] = 0 \\
\text{Var} \left[ \frac{\partial \ell}{\partial h_i^t} \right] = b \quad \forall_{i, t}
$$